## Notebook for the annotation transferring from Gut Cell Atlas to Joanito 2022 data

- **Developed by**: Anna Maguza
- **Institute of Computational Biology - Computational Health Centre - Helmholtz Munich**
- 28th November 2022

#### Import required modules

In [2]:
import scnym

In [3]:
import anndata
import numpy as np
import pandas as pd
import scanpy as sc

#### Setup Cells

In [4]:
%matplotlib inline

In [5]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi = 160, color_map = 'magma_r', dpi_save = 300, vector_friendly = True)

scanpy==1.6.0 anndata==0.7.4 umap==0.3.10 numpy==1.18.2 scipy==1.4.1 pandas==1.0.4 scikit-learn==0.22.2.post1 statsmodels==0.13.5 python-igraph==0.10.2 louvain==0.7.0 leidenalg==0.8.0


#### Read reference object

In [6]:
input_Normal_file = '/lustre/groups/talaveralopez/datasets/Colorectal_cancer/Raw_data/Normal Control Gut Cell Atlas/Full_obj_raw_counts_nosoupx_v2.h5ad'  # the file that will store the analysis results
output_Normal_file = '/lustre/groups/talaveralopez/workspace/anna.maguza/Processed_datasets/Gut_cell_atlas_three_donors_output.h5ad'  # the file that will store the analysis results

In [7]:
#Anndata upload
control_ad = sc.read_h5ad(input_Normal_file)
control_ad.X

<428469x33538 sparse matrix of type '<class 'numpy.float32'>'
	with 760344941 stored elements in Compressed Sparse Row format>

#### Quick Filtering of Gut Cell Atlas file

In [8]:
#Remove Pediatric Crohn Disease
control_ad_clean = control_ad[~control_ad.obs['Diagnosis'].isin(['Pediatric Crohn Disease']),:]
del control_ad
#Deleting the lymph node samples
control_ad_clean = control_ad_clean[~control_ad_clean.obs['Region'].isin(['lymph node']),:]
#Adding percentage of ribosomial genes
control_ad_clean.var['ribo'] = control_ad_clean.var_names.str.startswith(("RPS","RPL"))  # annotate the group of ribosomal genes as 'ribo'
sc.pp.calculate_qc_metrics(control_ad_clean, qc_vars=['ribo'], percent_top=None, log1p=False, inplace=True)
#Filtering by genes
control_ad_clean = control_ad_clean[control_ad_clean.obs.n_genes_by_counts < 5000, :]
control_ad_clean = control_ad_clean[control_ad_clean.obs.n_genes_by_counts > 200, :]
#Filtering by counts
control_ad_clean = control_ad_clean[control_ad_clean.obs.total_counts < 50000, :]

Trying to set attribute `.var` of view, copying.


#### Read in query

In [9]:
input_CRC_file = '/lustre/groups/talaveralopez/datasets/Colorectal_cancer/Raw_data/Joanito, 2022/Anndata Raw/joanito_2022_anndata_raw-2.h5ad'  # the file that will store the analysis results
output_CRC_file = '/lustre/groups/talaveralopez/workspace/anna.maguza/Processed_datasets/joanito_2022_anndata_output.h5ad'  # the file that will store the analysis results

In [10]:
#Anndata upload
Joanito_ad = sc.read_h5ad(input_CRC_file)
Joanito_ad.X

<373058x33287 sparse matrix of type '<class 'numpy.float32'>'
	with 741331835 stored elements in Compressed Sparse Row format>

#### Quick Filtering of Joanito file

In [11]:
#Filter out normal (non-tumor samles)
Joanito_ad_tumor = Joanito_ad[Joanito_ad.obs['sample.origin'] == "Tumor", :]
del Joanito_ad

In [12]:
#Filtering by quality metrics
#Calculate quality control metrics
sc.pp.calculate_qc_metrics(Joanito_ad_tumor, expr_type = 'counts', var_type='genes', percent_top = None, inplace=True)
#Filtering by genes
Joanito_ad_tumor = Joanito_ad_tumor[Joanito_ad_tumor.obs.n_genes_by_counts < 4000, :]
Joanito_ad_tumor = Joanito_ad_tumor[Joanito_ad_tumor.obs.n_genes_by_counts > 200, :]
#Filtering by counts
Joanito_ad_tumor = Joanito_ad_tumor[Joanito_ad_tumor.obs.total_counts < 20000, :]

Trying to set attribute `.obs` of view, copying.


#### Format reference for `scNym`

In [13]:
control_ad_clean

View of AnnData object with n_obs × n_vars = 372785 × 33538
    obs: 'Sample name', 'Diagnosis', 'Age', 'sample name', 'Region code', 'Fraction', 'Gender', 'Region', '10X', 'batch', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'doublet_scores', 'predicted_doublets', 'category', 'Age_group', 'Integrated_05', 'total_counts_ribo', 'pct_counts_ribo'
    var: 'gene_ids', 'feature_types', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [14]:
Joanito_ad_tumor

View of AnnData object with n_obs × n_vars = 150571 × 33287
    obs: 'cell.ID', 'nFeature_RNA', 'percent.mt', 'sample.ID', 'patient.ID', 'sample.origin', 'dataset_x', 'cell.type', 'iCMS', 'msi', 'dataset_y', 'Gender', 'Age at recruitment', 'MSS/MSI', 'Site', 'Sidedness', 'Group Stage', 'Stage TNM', 'iCMS.transcriptomic', 'iCMS.inferCNV', 'KRAS', 'BRAF', 'TP53', 'APC', 'PIK3CA', 'LymphNode', 'Normal', 'Tumor', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts'
    var: 'feature_types', 'genome', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'

In [15]:
# Make cell.ID column in Joanito_ad_tumor the index
Joanito_ad_tumor.obs.index = Joanito_ad_tumor.obs['cell.ID']
# Delete cell.ID column in Joanito_ad_tumor
del Joanito_ad_tumor.obs['cell.ID']

In [16]:
Joanito_ad_tumor.obs["Study"] = "Joanito, 2022"
Joanito_ad_tumor.obs['Study'] = Joanito_ad_tumor.obs['Study'].astype('category')
control_ad_clean.obs["Study"] = "Gut Cell Atlas"
control_ad_clean.obs['Study'] = control_ad_clean.obs['Study'].astype('category')
Joanito_ad_tumor.obs.rename(columns={"Age at recruitment": "Age"}, inplace=True)
Joanito_ad_tumor.obs.rename(columns={"percent.mt": "pct_counts_mt"}, inplace=True)
Joanito_ad_tumor.obs.rename(columns={"Site": "Region code"}, inplace=True)
Joanito_ad_tumor.obs['Region code'] = Joanito_ad_tumor.obs['Region code'].astype('category')

Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.


In [17]:
Joanito_ad_tumor.obs['domain_label'] = Joanito_ad_tumor.obs['sample.ID'].copy()
Joanito_ad_tumor.obs['domain_label'] = 'target_' + Joanito_ad_tumor.obs['domain_label'].astype(str)
Joanito_ad_tumor.obs['domain_label'] = Joanito_ad_tumor.obs['domain_label'].astype('category')
Joanito_ad_tumor.obs['domain_label'].cat.categories

Index(['target_CRC-JSC-S02-GIS-N-US', 'target_CRC-JSC-S02-GIS-T-US-1',
       'target_CRC-JSC-S02-GIS-T-US-2', 'target_CRC-JSC-S03-GIS-N-US-1',
       'target_CRC-JSC-S03-GIS-N-US-2', 'target_CRC-JSC-S03-GIS-T-US-1',
       'target_CRC-JSC-S03-GIS-T-US-2', 'target_CRC-JSC-S04-GIS-T-US-1',
       'target_CRC-JSC-S04-GIS-T-US-2', 'target_CRC-JSC-S05-GIS-T-US-1',
       'target_CRC-JSC-S05-GIS-T-US-2', 'target_CRC-JSC-S06-GIS-T-US-1',
       'target_CRC-JSC-S06-GIS-T-US-2', 'target_CRC-JSC-S07-GIS-T-US-1',
       'target_CRC-JSC-S07-GIS-T-US-2', 'target_CRC-JSC-S08-GIS-T-US-1',
       'target_CRC-JSC-S08-GIS-T-US-2', 'target_CRC-JSC-S10-GIS-T-US-1',
       'target_CRC-JSC-S10-GIS-T-US-2', 'target_CRC-JSC-S12-GIS-T-US-1',
       'target_CRC-JSC-S12-GIS-T-US-2', 'target_CRC-JSC-S13-GIS-T-US-1',
       'target_CRC-JSC-S13-GIS-T-US-2', 'target_CRC-JSC-S14-GIS-T-US-1',
       'target_CRC-JSC-S14-GIS-T-US-2', 'target_CRC-JSC-S15-GIS-N-US-1',
       'target_CRC-JSC-S15-GIS-N-US-2', 'target_CRC-J

In [18]:
Joanito_ad_tumor.obs['cell_states'] = 'Unlabeled'
Joanito_ad_tumor.obs['cell_states'] = Joanito_ad_tumor.obs['cell_states'].astype('category')

In [19]:
Joanito_ad_tumor.obs

,nFeature_RNA,pct_counts_mt,sample.ID,patient.ID,sample.origin,dataset_x,cell.type,iCMS,msi,dataset_y,...,LymphNode,Normal,Tumor,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,Study,domain_label,cell_states
cell.ID,,,,,,,,,,,,,,,,,,,,,
CRC16_MUX8563_AAAGATGCAGAAGCAC-1,4759,23.734351,MUX8563,CRC2794,Tumor,CRC-SG1,Epithelial,iCMS2,MSS,CRC-SG1,...,0.0,1.0,4.0,1531,7.334329,3259.0,8.089482,"Joanito, 2022",target_MUX8563,Unlabeled
CRC16_MUX8563_ACAGCCGGTCTCTTAT-1,2499,11.020450,MUX8563,CRC2794,Tumor,CRC-SG1,Epithelial,iCMS2,MSS,CRC-SG1,...,0.0,1.0,4.0,1686,7.430707,3771.0,8.235361,"Joanito, 2022",target_MUX8563,Unlabeled
CRC16_MUX8563_ACATACGGTTACGTCA-1,3745,17.618134,MUX8563,CRC2794,Tumor,CRC-SG1,Epithelial,iCMS2,MSS,CRC-SG1,...,0.0,1.0,4.0,3080,8.033009,7321.0,8.898639,"Joanito, 2022",target_MUX8563,Unlabeled
CRC16_MUX8563_ACATGGTGTCCATGAT-1,6874,17.782717,MUX8563,CRC2794,Tumor,CRC-SG1,Epithelial,iCMS2,MSS,CRC-SG1,...,0.0,1.0,4.0,1383,7.232733,2459.0,7.807917,"Joanito, 2022",target_MUX8563,Unlabeled
CRC16_MUX8563_ACGAGGACATCTGGTA-1,5116,18.923530,MUX8563,CRC2794,Tumor,CRC-SG1,Epithelial,iCMS2,MSS,CRC-SG1,...,0.0,1.0,4.0,1562,7.354362,3136.0,8.051023,"Joanito, 2022",target_MUX8563,Unlabeled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KUL5_EXT127_TTGGAACGTGAAAGAG-1,1144,8.299758,EXT127,SC044,Tumor,KUL5,T_NK,nan,nan,KUL5,...,0.0,2.0,3.0,3298,8.101375,8446.0,9.041567,"Joanito, 2022",target_EXT127,Unlabeled
KUL5_EXT127_TTGGCAACATGGTAGG-1,1235,2.585657,EXT127,SC044,Tumor,KUL5,PlasmaB,nan,nan,KUL5,...,0.0,2.0,3.0,2786,7.932721,6576.0,8.791334,"Joanito, 2022",target_EXT127,Unlabeled
KUL5_EXT127_TTTCCTCCATTACGAC-1,833,6.911582,EXT127,SC044,Tumor,KUL5,T_NK,nan,nan,KUL5,...,0.0,2.0,3.0,2077,7.639161,4410.0,8.391857,"Joanito, 2022",target_EXT127,Unlabeled


#### Format reference for `scNym`

In [20]:
control_ad_clean.obs['cell_states'] = control_ad_clean.obs['category'].copy()
control_ad_clean.obs['cell_states'] = control_ad_clean.obs['cell_states'].astype('category')

In [22]:
control_ad_clean.obs['domain_label'] = control_ad_clean.obs['sample name'].copy()
control_ad_clean.obs['domain_label'] = 'train_' + control_ad_clean.obs['domain_label'].astype(str)
control_ad_clean.obs['domain_label'] = control_ad_clean.obs['domain_label'].astype('category')
control_ad_clean.obs['domain_label'].cat.categories

Index(['train_A26-CAE-0-SC-1', 'train_A26-SCL-0-SC-1', 'train_A26-TCL-0-SC-1',
       'train_A30-SCL-6-SC-45N-1', 'train_A30-SCL-6-SC-45N-2',
       'train_A30-SCL-6-SC-45P-1', 'train_A30-SCL-6-SC-45P-2',
       'train_A32-APD-0-SC-45N-1', 'train_A32-APD-0-SC-45P-1',
       'train_A32-CAE-0-SC-45N-1',
       ...
       'train_T036-TIL-SC-EPCAMN', 'train_T036-TIL-SC-EPCAMP',
       'train_T057-TIL-SC-EPCAMP', 'train_T110-TIL-SC-EPCAMN',
       'train_T110-TIL-SC-EPCAMP', 'train_T160-TIL-SC-EPCAMP',
       'train_T161-TIL-SC-EPCAMP', 'train_T182-TIL-SC-EPCAMP',
       'train_T44-TIL-SC-EPCAMP', 'train_nan'],
      dtype='object', length=110)

In [23]:
control_ad_clean.obs['cell_states'].value_counts()

Mesenchymal        155794
Epithelial         129929
Neuronal            17594
T cells             17394
Plasma cells        16445
Endothelial         13082
B cells             11381
Myeloid              9820
Red blood cells      1346
Name: cell_states, dtype: int64

#### Merge two objects 

In [24]:
REF_QUERY = Joanito_ad_tumor.concatenate(control_ad_clean, batch_key = 'object', batch_categories = ['query', 'reference'], join = 'inner')
REF_QUERY

AnnData object with n_obs × n_vars = 523356 × 33264
    obs: 'nFeature_RNA', 'pct_counts_mt', 'sample.ID', 'patient.ID', 'sample.origin', 'dataset_x', 'cell.type', 'iCMS', 'msi', 'dataset_y', 'Gender', 'Age', 'MSS/MSI', 'Region code', 'Sidedness', 'Group Stage', 'Stage TNM', 'iCMS.transcriptomic', 'iCMS.inferCNV', 'KRAS', 'BRAF', 'TP53', 'APC', 'PIK3CA', 'LymphNode', 'Normal', 'Tumor', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'Study', 'domain_label', 'cell_states', 'Sample name', 'Diagnosis', 'sample name', 'Fraction', 'Region', '10X', 'batch', 'n_genes', 'total_counts_mt', 'doublet_scores', 'predicted_doublets', 'category', 'Age_group', 'Integrated_05', 'total_counts_ribo', 'pct_counts_ribo', 'object'
    var: 'feature_types', 'genome-query', 'n_cells_by_counts-query', 'mean_counts-query', 'log1p_mean_counts-query', 'pct_dropout_by_counts-query', 'total_counts-query', 'log1p_total_counts-query', 'n_cells_by_counts-reference', 'mean_counts-r

#### Clean up object

- Clean up `adata.obs`

In [25]:
REF_QUERY.obs.drop(REF_QUERY.obs.columns.difference(['domain_label','cell_states', 'object', "cell.type", "Region code"]), 1, inplace = True)
REF_QUERY

AnnData object with n_obs × n_vars = 523356 × 33264
    obs: 'cell.type', 'Region code', 'domain_label', 'cell_states', 'object'
    var: 'feature_types', 'genome-query', 'n_cells_by_counts-query', 'mean_counts-query', 'log1p_mean_counts-query', 'pct_dropout_by_counts-query', 'total_counts-query', 'log1p_total_counts-query', 'n_cells_by_counts-reference', 'mean_counts-reference', 'pct_dropout_by_counts-reference', 'total_counts-reference', 'gene_ids-reference', 'ribo-reference'

- Clean up `adata.var`

In [26]:
REF_QUERY.var.drop(REF_QUERY.var.columns.difference(['mt']), 1, inplace = True)
REF_QUERY

AnnData object with n_obs × n_vars = 523356 × 33264
    obs: 'cell.type', 'Region code', 'domain_label', 'cell_states', 'object'

In [27]:
del Joanito_ad_tumor, control_ad_clean

In [31]:
REF_QUERY.obs['cell.type'].value_counts()

T_NK            50441
Epithelial      32181
McDC            17895
PlasmaB         16958
Fibroblast      16240
B                7674
Endothelial      5037
Neutrophils      1692
Mast             1294
Entericglial      858
pDC               301
Name: cell.type, dtype: int64

In [32]:
REF_QUERY.obs['cell_states'].value_counts()

Mesenchymal        155794
Unlabeled          150571
Epithelial         129929
Neuronal            17594
T cells             17394
Plasma cells        16445
Endothelial         13082
B cells             11381
Myeloid              9820
Red blood cells      1346
Name: cell_states, dtype: int64

#### LogP1 normalization

In [29]:
#Log normalization
sc.pp.log1p(REF_QUERY)

#### Train reference with `scNym`

In [30]:
scnym.api.scnym_api(
    adata = REF_QUERY,
    task = 'train',
    groupby = 'cell_states',
    domain_groupby='domain_label',
    out_path = '/lustre/groups/talaveralopez/workspace/anna.maguza/Processed_datasets/scnym_training_on_all_GCA_donors',
    config = 'new_identity_discovery',
)

No CUDA device found.
Computations will be performed on the CPU.
Add a CUDA compute device to improve speed dramatically.

150571 unlabeled observations found.
Using unlabeled data as a target set for semi-supervised, adversarial training.

training examples:  (372785, 33264)
target   examples:  (150571, 33264)
X:  (372785, 33264)
y:  (372785,)
Using user provided domain labels.
Found 110 source domains and 97 target domains.
Not weighting classes and not balancing classes.
Found 207 unique domains.
Using MixMatch for semi-supervised learning
Scaling ICL over 100 epochs, 0 epochs for burn in.
Scaling ICL over 20 epochs, 0 epochs for burn in.
Using a Domain Adaptation Loss.
Training...


KeyboardInterrupt: 